## Public-Key Encryption

To solve the Mallory-in-the-middle problem, we will need to explore a modern advance in cryptography known as **public-key cryptography**. Stream and block ciphers encrypt and decrypt symmetrically. One key is used for both encryption and decryption. Public-key cryptosystems use a **keypair**. They have two keys that work as partners. If one key is used to encrypt plaintext, only its partner key can be used to decrypt the cyphertext.

Typically, one key in the keypair is public. The other is kept private. Let's say Alice has a keypair $e$ and $d$. She keeps $e$ secret and lets everyone know $d$. Anyone can use $d$ to encrypt messages to send to Alice, but only Alice can decrypt them with $e$. This is nice. Instead of exchanging keys with each person individually, Alice can exchange one key with the whole world. Convenient, but not revolutionary.

The most powerful use for public-key encryption uses the keys in the opposite direction. Alice can encrypt a message using her private key $e$. Anyone can decrypt it with her public key $d$. What's the point? Alice is sending out a message that everyone can read. Why doesn't she just send out the plaintext? Because **only** Alice has private key $e$. If Bob can decrypt a message using Alice's public key $d$, Bob **knows** the message came from Alice and **not** Mallory. No Mallory-in-the-middle!

Public-key cryptosystems can be a bit counter-intuitive. Let's look at an example.

## RSA Setup

RSA is a public-key cryptosystem that relies on the difficulty of factoring for security. There's quite a bit of setup involved to make RSA work, so you'll have to be a bit patient. First, we need to choose two prime numbers $p$ and $q$. These should be large, but let's use small numbers in an example $p=137$ and $q=163$. We should keep $p$ and $q$ secret!

In [ ]:
p=137
q=163
n=p*q

Next, we need to compute $\phi(n)$. This is very easy. Since we know $n=pq$, and $p$ and $q$ both prime numbers, $\phi(n)=(p-1)(q-1)$. Without knowing the factorization of $n$, computing $\phi(n)$, as far as we know, impossible.

In [ ]:
phi=(p-1)*(q-1)

Now that we know $phi(n)$, we randomly select the first key $e$ in our key pair from the numbers $1$ through $\phi(n)$. Let's say our random number generator chooses 13.

In [ ]:
e=13

To find the matching key $d$ and complete our key pair, we need to find the multiplicative inverse of $e \bmod \phi(n)$.

In [ ]:
import gmpy2
d=int(gmpy2.invert(e,phi))
print("Our keypair: ",e,d)

We now have an RSA keypair $e$ and $d$. We can broadcast to everyone our public key $d$ and $n$. We keep our private key $e$ a secret.

## RSA Encryption

If someone wants to send a message $M=2112$ to us, they can encrypt it using our public key by computing $C=M^d \bmod n$.

In [ ]:
M=2112
C=pow(M,d,n)
print(C)

 They can then broadcast ciphertext $C$ to world. Only we can decrypt it, because we have the private key $e$. We decrypt the message by computing $M^e \bmod n$.

In [ ]:
pow(C,e,n)

A part of your brain might think that the public key $d$ could somehow be used to decrypt the message. Let me assure you that this does not work. Using it to decrypt produces nothing meaningful.

In [ ]:
pow(C,d,n)

## Attacking RSA

Given ciphertext $C$, the public key $d$ and modulus $n$, what can an adversary do? They need to find $e$ to decrypt $C$. Finding $e$ requires finding the multiplicative inverse of $d \bmod \phi(n)$. To make any progress, the attacker needs to know $\phi(n)$. There is no known way to compute $\phi(n)$ without factoring $n$. In the end, the direct attack boils down factoring $n$. For a sufficently large enough $n$, this could take decades, centuries, or even millennia. Unless, of course, someone discovers a faster factoring algorithm some day.

This is not to say RSA is perfect. It's a very complicated and counterintuitive system. Unless people are very careful, RSA can be fragile. Researchers have made many clever and subtle attacks against it in the past. Let's learn about a couple examples before we get to the exercises.

## Common Primes Attack

This was an attack used against **real** RSA cryptosystems. We'll discuss it using a simplified example, but this isn't a toy attack. It's absolutely practical.

Let's say 5 different people generate their own RSA cryptosystem. All of them independently find their own $p$ and $q$ and compute $n$.

In [ ]:
n1=0x854b861409f3cfdf6df511843c0ba66e78dfc07d7ef93118a6aa87aedf87ee3
n2=0x3fe6cc0047af497250d22d99e40b6b10ae85f8d21914ff286198f49e3d5987d5
n3=0xf527d2326eaec0150a945288b20c1a15896681b814e10737e2a5f16a29ed535
n4=0x2e3129aaa1c9bd97e72ca6c17c1eb37a552d15694a81aba1018b38420ba504c1
n5=0x1c121335f3f17d3eaa35eee5f7b3b8144e0e0d6b2e06f9618379e1dd3637c03d

ns=[n1,n2,n3,n4,n5]

Each of these 5 ns is 256-bits long. Factoring even one of them would take a very long time. I'm going to factor all of them very quickly.

In [ ]:
from itertools import combinations
from math import gcd

for a,b in combinations(ns,2):
    print(hex(gcd(a,b)))

In [ ]:
n1==0x696dabf8b4adeab455fd67d67c70c17d*0x143aa55a43690830a44283c45a422fdf
n2==0x696dabf8b4adeab455fd67d67c70c17d*0x9b2a3c6cbac1d09967c90fde48f41be3
n3==0x696dabf8b4adeab455fd67d67c70c17d*0x25348a6b37393f3bb791c7075639377c
n4==0x442924573be36dd7a4d076e54a7335c1*0xad7d4b306301f8a14f3d27f38eeb8f01
n5==0x442924573be36dd7a4d076e54a7335c1*0x696dabf8b4adeab455fd67d67c70c17d

What happened? Some of these people used the same $p$ or $q$ to generate their $n$. That's a big mistake. Why? Factoring is hard, but finding the GCD is easy, remember? My code above finds the GCD of all of the pairs of ns. The factors fall right out.

This kind of thing shouldn't happen by accident. We'd have to be very unlucky to pick identical random large numbers by chance. But this **did** happen, and was [discovered by researchers in 2012](https://factorable.net/paper.html). What caused it? Partially, it was caused bad random number generation. The entropy pool used to generate RSA keys in some cheap devices wasn't truly random.

## Bit Flip Attack

Another somewhat counter-intuitive attack against RSA is a bit flip attack. There are techniques (e.g. Rowhammer) that can corrupt memory. Mallory might not be able to inject code onto Alice's computer, but she might be able cause a bit to flip. Again, this is a **real** attack against RSA [discovered by researchers in 2016](https://www.usenix.org/conference/usenixsecurity16/technical-sessions/presentation/razavi).

So what? What's the importance of one bit? If Mallory gets lucky enough to flip one random bit in $n$, she can completely ruin RSA.

Here's an n that's the product of two large primes. It is 256 bits long. Factoring $n$ to find $p$ and $q$ would take a long time.

In [ ]:
n=0x22197ebaff43a591f531b4eb831d62402872f14b78b511a061e08feefa214cc9

Now I will flip the 21st bit in $n$. There's no significance to 21. I chose it at random.

In [ ]:
def flipBit(n,i):
    bits=bin(n)[2:]
    if bit=='1':
        bits=bits[:flip]+'0'+bits[flip+1:]
    else:
        bits=bits[:flip]+'1'+bits[flip+1:]
    return int(bits,2)

n=0x22197ebaff43a591f531b4eb831d62402872f14b78b511a061e08feefa214cc9
n=flipBit(n,21)

from sympy import ntheory
print(ntheory.factorint(n))

With one bit flipped, $n$ is no longer the product of two large primes. Now, it has much smaller factors. Factoring $n$ takes no time at all. The entire RSA cryptosystem falls apart because **one** bit flipped.

## Exercises
Below are the exercises for this section. If you have any difficulties, don't hesitate to ask for help. Don't forget to take breaks!

1) My public key $d$ and modulus $n$ are below. You receive two messages M1 and M2. I also encrypted M with my private key and sent you the result, S. Which message did I send you?

In [ ]:
n=0xf3a7f6d00ff04ef2b347b9acd499b1298531b2e5ef12ff962fe8af5574a3a3d
d=0xf57bc92839b85b1f8218d60e45aecc8f60db46ebc6de02c26ffda687f5632d

M1=0x546869732069732074686520636f727265637420616e737765722e
M2=0x54686973206973207468652077726f6e6720616e737765722e

S=0x599de306d8b22e9fb33174c1b117ac705a0b84cee77ee8344fd59344b236235

2) Using my same public key and modulus from the previous exercise, encrypt the message M below.

In [ ]:
n=0xf3a7f6d00ff04ef2b347b9acd499b1298531b2e5ef12ff962fe8af5574a3a3d
d=0xf57bc92839b85b1f8218d60e45aecc8f60db46ebc6de02c26ffda687f5632d

M=0x544f5020534543524554204d45535341474521

3) Here are 2 RSA cryptosystems used to encrypt 2 messages into ciphertext $C_1$ and $C_2$. Can you decrypt the messages?

In [ ]:
n1=0x79edc7fc3719926cb708276aed8c8a61a37dd49ad454f0ca4dee277ed3df576d7ee60c39b1950fd01fb063f0798b50bc09854d1f88214803df11a97c8710416b
d1=0x7350d38539bab9fd2ffb51d53f609e54d8975f4ebffbf6c4f50ca8258b523435872c9e2f47544a8bfeb36c83d1f9c27601da9547aa1c446043e09c0386cdf319
C1=0x6857d5233a627865e78afd3cdfda66145a721f0589d132122680eaa3627ca845ffd80a852613932bddaa28f65509fdfd3567cb25c6fef9642a06744c81cf2ea3

n2=0xb9dbb0246726719c6e70e68913886df95faa37f47bfc7d9de005d2ed8020592a76244db8854e44fa122fca052d540210467d3003fa6f129d68726ba864b7f19
d2=0x8bed02013f67846ac0ad0eedad9853acf48115ea5ee911fe48fec44a617a16711c53462e7ac3bcfee3e7bad2f041ed98b11bd263f12ad281e0a5b3aec1997eb
C2=0x77e6a5c76f5c24e15f9f299b93e1f0e6f8e6d355d0bfb14c8bd67710654c3a5f712beefd60d5ea66d2a5cf2f35dcd7d66612d707b429bd65c805dade7ea1718